In [17]:
import pandas as pd 
import yfinance as yf 
import numpy as np 
import datetime as dt 
import matplotlib.pyplot as plt 

In [27]:
tsla_df = yf.download('TSLA', start='2018-08-06', end='2023-09-01') 
tsla_df.index = pd.to_datetime(tsla_df.index)
tsla_df = tsla_df.sort_index(ascending=False)
# tsla_df

close_df = tsla_df['Close']
close_df.columns = ['Close']
close_df = close_df.reset_index().rename(columns={"Date": "date"})

rv_df = close_df.copy()

rv_df['Log_return'] = np.log(close_df['Close'] / close_df['Close'].shift(1))

def get_RV(row, log_returns):
    window = 30
    idx = log_returns.index.get_loc(row.name)
    if idx < window - 1:
        return np.nan
    window_returns = log_returns.iloc[idx - window + 1 : idx + 1]
    return window_returns.std() * np.sqrt(252)

rv_df['RV'] = rv_df.apply(get_RV, axis=1, log_returns=rv_df['Log_return'])

# remove rows with NA value in the RV column
rv_df = rv_df.dropna(subset=['RV'])
rv_df = rv_df.reset_index(drop=True)

# log-transform the RV values
logrv_df = rv_df.copy()
logrv_df['Log_RV'] = np.log(rv_df['RV'])

logrv_df


[*********************100%***********************]  1 of 1 completed


,date,Close,Log_return,RV,Log_RV
0,2023-07-21,260.019989,-0.034176,0.463274,-0.769436
1,2023-07-20,262.899994,0.011015,0.456283,-0.784642
2,2023-07-19,291.260010,0.102443,0.542900,-0.610830
3,2023-07-18,293.339996,0.007116,0.542882,-0.610863
4,2023-07-17,290.380005,-0.010142,0.491823,-0.709636
...,...,...,...,...,...
1243,2018-08-10,23.699333,-0.002585,0.525701,-0.643022
1244,2018-08-09,23.496668,-0.008588,0.526892,-0.640759
1245,2018-08-08,24.689333,0.049513,0.541407,-0.613584
1246,2018-08-07,25.304667,0.024618,0.543510,-0.609707
